In [317]:
import pandas as pd
from random import shuffle
import numpy as np

# chatGPT generation for BestLead

In [5]:
import random

SUITS = ['S', 'H', 'D', 'C']
RANKS = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

def has_xM(hand, x=5):
    suit_count = {suit: 0 for suit in SUITS}
    for card in hand:
        suit_count[card[0]] += 1
    return suit_count['S'] >= x or suit_count['H'] >= x

def has_xm(hand, x=6):
    suit_count = {suit: 0 for suit in SUITS}
    for card in hand:
        suit_count[card[0]] += 1
    return suit_count['C'] >= x or suit_count['D'] >= x

def has_xcard(hand, x=7):
    suit_count = {suit: 0 for suit in SUITS}
    for card in hand:
        suit_count[card[0]] += 1
    return suit_count['S'] >= x or suit_count['H'] >= x or suit_count['C'] >= x or suit_count['D'] >= x

def generate_pbn(num=1):
    # Define card values for High Card Points (HCP) and order for sorting
    CARD_VALUES = {
        '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, 'T': 0,
        'J': 1, 'Q': 2, 'K': 3, 'A': 4
    }

    # Define the suits and their order for sorting
    

    def calculate_hcp(hand):
        """Calculate the High Card Points (HCP) of a hand."""
        return sum(CARD_VALUES[card[1]] for card in hand)

    def is_balanced(hand):
        """Check if a hand is balanced (no more than one doubleton)."""
        suit_count = {suit: 0 for suit in SUITS}
        for card in hand:
            suit_count[card[0]] += 1
        distribution = sorted(suit_count.values())
        return distribution == [2, 3, 4, 4] or distribution == [3, 3, 3, 4] or distribution == [2,3,3,5]

    def generate_hand():
        """Generate a random hand of 13 cards."""
        deck = [suit + rank for suit in SUITS for rank in RANKS]
        random.shuffle(deck)
        return deck[:13]

    def format_hand(hand):
        """Format a hand in the specified format with sorted cards."""
        suit_groups = {suit: [] for suit in SUITS}
        for card in hand:
            suit_groups[card[0]].append(card[1])
        
        # Sort cards in each suit according to the defined rank order
        sorted_hand = [''.join(sorted(suit_groups[suit], key=lambda x: RANKS.index(x), reverse=False)) for suit in SUITS]
        
        return '.'.join(sorted_hand)

    def generate_pbn():
        """Generate PBN format hands with West balanced and 15-17 HCP, East with at most 7 HCP."""
        def condition_1nt(north_hand, south_hand, west_hand, east_hand):
            west_hcp=calculate_hcp(west_hand)
            east_hcp=calculate_hcp(east_hand)
            return 15 <= west_hcp <= 17 and east_hcp <= 7 and is_balanced(west_hand) and not has_xM(east_hand, x=5) and not has_xm(east_hand, x=6) and not has_xcard(north_hand, x=7) and not has_xcard(south_hand, x=7)
        
        def condition_1nt_3nt(north_hand, south_hand, west_hand, east_hand):
            west_hcp=calculate_hcp(west_hand)
            east_hcp=calculate_hcp(east_hand)
            return 15 <= west_hcp <= 17 and 9 <= east_hcp <= 15 and is_balanced(west_hand) and not has_xM(east_hand, x=4) and not has_xm(east_hand, x=6) and not has_xcard(north_hand, x=7) and not has_xcard(south_hand, x=7)
        
        while True:
            # Generate hands for all players
            deck = [suit + rank for suit in SUITS for rank in RANKS]
            random.shuffle(deck)
            west_hand = deck[:13]
            east_hand = deck[13:26]
            north_hand = deck[26:39]
            south_hand = deck[39:52]
            
            
            if condition_1nt_3nt(north_hand, south_hand, west_hand, east_hand):                 ####CONFIG!!!!
                break

        # Format the hands
        hands = {
            'N': format_hand(north_hand),
            'E': format_hand(east_hand),
            'S': format_hand(south_hand),
            'W': format_hand(west_hand),
        }

        return f"N:{hands['N']} E:{hands['E']} S:{hands['S']} W:{hands['W']}"

    # Generate a PBN hand
    pbn_hands = [generate_pbn() for _ in range(num)]
    pbn_hand = pbn_hands[0]

    # Save to a .txt file
    with open("bridge_hand.txt", "w") as file:
        for pbn_hand in pbn_hands:
            file.write(pbn_hand + "\n")


    print("PBN hand saved to bridge_hand.txt:")
    print(pbn_hand)
    return





# Generating more hands given the player on the lead's hand

In [9]:
import random
import subprocess

def generate_hands_given_leader(num=1000):
    # Define card values for High Card Points (HCP) and order for sorting
    CARD_VALUES = {
        '2': 0, '3': 0, '4': 0, '5': 0, '6': 0, '7': 0, '8': 0, '9': 0, 'T': 0,
        'J': 1, 'Q': 2, 'K': 3, 'A': 4
    }

    # Define the suits and their order for sorting
    SUITS = ['S', 'H', 'D', 'C']
    RANKS = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']

    def calculate_hcp(hand):
        """Calculate the High Card Points (HCP) of a hand."""
        return sum(CARD_VALUES[card[1]] for card in hand)

    def is_balanced(hand):
        """Check if a hand is balanced (no more than one doubleton)."""
        suit_count = {suit: 0 for suit in SUITS}
        for card in hand:
            suit_count[card[0]] += 1
        distribution = sorted(suit_count.values())
        return distribution == [2, 3, 4, 4] or distribution == [3, 3, 3, 4] or distribution == [2,3,3,5]
    
    def longer(hand, longer_suit, shorter_suit, strictly=False):
        suit_count = {suit: 0 for suit in SUITS}
        for card in hand:
            suit_count[card[0]] += 1
        if strictly:
            return suit_count[longer_suit] > suit_count[shorter_suit]
        return suit_count[longer_suit] >= suit_count[shorter_suit]

    def format_hand(hand):
        """Format a hand in the specified format with sorted cards."""
        suit_groups = {suit: [] for suit in SUITS}
        for card in hand:
            suit_groups[card[0]].append(card[1])
        
        # Sort cards in each suit according to the defined rank order
        sorted_hand = [''.join(sorted(suit_groups[suit], key=lambda x: RANKS.index(x), reverse=False)) for suit in SUITS]
        
        return '.'.join(sorted_hand)

    def parse_hand(hand_string):
        """Parse the hand from the string format and return hands for N, E, S, W."""
        hands = hand_string.split()
        north_hand = hands[0].split(':')[1]
        east_hand = hands[1].split(':')[1]
        south_hand = hands[2].split(':')[1]
        west_hand = hands[3].split(':')[1]
        
        return north_hand, east_hand, south_hand, west_hand

    def condition_1nt(north_hand, south_hand, west_hand, east_hand):
        west_hcp=calculate_hcp(west_hand)
        east_hcp=calculate_hcp(east_hand)
        #print(east_hand)
        return 15 <= west_hcp <= 17 and east_hcp <= 7 and is_balanced(west_hand) and not has_xM(east_hand, x=5) and not has_xm(east_hand, x=6) and not has_xcard(north_hand, x=7) and not has_xcard(south_hand, x=7)
        
    def condition_1nt_3nt(north_hand, south_hand, west_hand, east_hand):
        west_hcp=calculate_hcp(west_hand)
        east_hcp=calculate_hcp(east_hand)
        return 15 <= west_hcp <= 17 and 9 <= east_hcp <= 15 and is_balanced(west_hand) and not has_xM(east_hand, x=4) and not has_xm(east_hand, x=6) and not has_xcard(north_hand, x=7) and not has_xcard(south_hand, x=7)
    
    def condition_Gregor(north_hand, south_hand, west_hand, east_hand):
        west_hcp=calculate_hcp(west_hand)
        east_hcp=calculate_hcp(east_hand)
        west_conditions = 10 <= west_hcp <= 12 and not has_xM(west_hand, x=4) and not has_xcard(west_hand, x=7)
        east_conditions = 14 <= east_hcp <= 16 and  is_balanced(east_hand) and not has_xM(east_hand, x=5) and longer(east_hand, 'C', 'D', strictly=True)
        return west_conditions and east_conditions and not has_xcard(south_hand, x=7)

    def generate_east_west_hand(remaining_cards, current_north_hand):
        """Generate a valid West hand with 15-17 HCP and balanced."""
        #print("remaining_cards", remaining_cards)
        while True:
            random.shuffle(remaining_cards)
            west_hand = remaining_cards[:13]
            east_hand = remaining_cards[13:26]
            south_hand = remaining_cards[26:39]
            
            if condition_Gregor(current_north_hand, south_hand, west_hand, east_hand):                 ####CONFIG!!!!
                #print("out now")
                return west_hand, east_hand

    def generate_hands(north_hand, num=1000):
        """Generate 1000 valid hands based on the provided North hand."""
        results = []
        full_deck = [suit + rank for suit in SUITS for rank in RANKS]
        north_hand_cards = north_hand.split('.')
        
        current_north_hand = []
        for idx, suit in enumerate(north_hand_cards):
            current_north_hand+=[SUITS[idx] + rank for rank in suit if rank in RANKS]
            
        
        print(current_north_hand)
        
        # Remove North hand from the deck
        for card in current_north_hand:
            full_deck.remove(card)

        for _ in range(num):
            # Generate West hand and shuffles the deck
            west_hand, east_hand = generate_east_west_hand(full_deck, current_north_hand)

            # Get remaining cards for East and South
            south_hand = [card for card in full_deck if card not in west_hand+east_hand]
            
            # Format the hands
            hands = {
                'N': format_hand(current_north_hand),
                'E': format_hand(east_hand),
                'S': format_hand(south_hand),
                'W': format_hand(west_hand),
            }
            
            # Append formatted hand to results
            results.append(f"N:{north_hand} E:{hands['E']} S:{hands['S']} W:{hands['W']}")
        
        return results

    # Load the hand from a .txt file
    with open("bridge_hand.txt", "r") as file:
        hand_string = file.readline().strip()

    # Parse the hand
    north_hand, east_hand, south_hand, west_hand = parse_hand(hand_string)
    #print(north_hand)

    # Generate 1000 hands based on the North hand
    generated_hands = generate_hands(north_hand, num=num)

    # Save generated hands to another .txt file
    with open("generated_hands.txt", "w") as file:
        for hand in generated_hands:
            file.write(hand + "\n")  # Add newline at the end of each hand

    print("Generated hands saved to generated_hands.txt.")
    return

#generate_hands_given_leader(1000)

def run_bash_script(script_path):
    """Run a Bash script given its full path."""
    try:
        # Run the script and wait for it to complete
        result = subprocess.run(['bash', script_path], check=True, text=True, capture_output=True)
        
        # Print the output of the script
        print("Output:\n", result.stdout)
        
        # Return the exit code
        return result.returncode
    except subprocess.CalledProcessError as e:
        print("Error executing the script:", e)
        print("Return code:", e.returncode)
        print("Output:\n", e.output)
        return e.returncode

# Example usage
script_path = "/home/kstef/Desktop/SDS/dds/examples/script.sh"  # Replace with the full path to your Bash script
#exit_code = run_bash_script(script_path)
#print("Script exited with code:", exit_code)


# Parsing dd output

In [10]:
import csv

score_map_1nt = { 13-i : (-50*(7-i) if i<7 else 90+30*(i-7)) for i in range(14) }
score_map_3nt = { 13-i : (-50*(9-i) if i<9 else 400+30*(i-9)) for i in range(14) }
imp_conversion = [0, 20, 50, 90, 130, 170, 220, 270, 320, 370, 430, 500, 600, 750, 900, 1100, 1300, 1500, 1750, 2000, 2250, 2500, 3000, 3500, 4000]


def parse_solutions(category_name, score_map=score_map_1nt, idx=1, tricks_to_make=9):
    def parse_csv_to_dict(filename, score_map=score_map):
        """Parse the CSV file and return a dictionary of cards and their scores."""
        card_score_map = {} 
        card_trick_map = {}
        card_MP_map = {}
        card_made_map = {}
        card_imploss_map = {}
        
        def add_MP(l, card_MP_map=card_MP_map):
            for card,val in l:
                if card in card_MP_map.keys():
                    card_MP_map[card] += (sum([2 if i[1]<val else (1 if i[1]==val else 0) for i in l])-1)/24
                else:
                    card_MP_map[card] = (sum([2 if i[1]<val else (1 if i[1]==val else 0) for i in l])-1)/24
            return card_MP_map
        
        def add_imploss(l, card_imploss_map=card_imploss_map):
            def find_imploss(best_score, score):
                idx=0
                while best_score <= score-imp_conversion[idx]:
                    idx+=1
                return idx-1
            best=max(l, key=lambda x: x[1])
            scores = [[i[0], score_map[i[1]]] for i in l]
            best_score=score_map[best[1]]
            
            for card,score in scores:
                if card in card_imploss_map.keys():
                    card_imploss_map[card] += find_imploss(best_score, score)
                else:
                    card_imploss_map[card] = find_imploss(best_score, score)
            return card_imploss_map
        
        with open(filename, mode='r', newline='') as file:
            reader = csv.reader(file)
            counter = 1
            next(reader)  # Skip the header row
            current_hand = []
            for row in reader:
                if row[1]=='card':
                    card_MP_map = add_MP(current_hand)
                    card_imploss_map = add_imploss(current_hand)
                    counter+=1
                    current_hand = []
                    continue
                # Extract suit, rank, and score
                suit = row[2]
                rank = row[3]
                rank2 = row[4]
                #print(rank2)
                score = score_map[int(row[5]) if row[5] else 0]  # Default to 0 if score is empty
                
                # Construct card representation
                card = f"{suit}{rank}" if rank else suit  # Handle cases with empty rank
                #print(counter, row)
                current_hand.append([card, int(row[5])])
                
                # Add to the dictionary
                if card in card_score_map.keys():
                    card_score_map[card] += score
                    card_made_map[card] += (1 if int(row[5])<=13-tricks_to_make else 0)
                    card_trick_map[card] += 13-int(row[5])
                else:
                    card_score_map[card] = score
                    card_made_map[card] = (1 if int(row[5])<=13-tricks_to_make else 0)
                    card_trick_map[card] = 13-int(row[5])
                    
                for rank1 in rank2:
                    card = f"{suit}{rank1}" if rank1 else suit
                    current_hand.append([card, int(row[5])])
                    if card in card_score_map.keys():
                        card_score_map[card] += score
                        card_made_map[card] += (1 if int(row[5])<=13-tricks_to_make else 0)
                        card_trick_map[card] += 13-int(row[5])
                    else:
                        card_score_map[card] = score
                        card_made_map[card] = (1 if int(row[5])<=13-tricks_to_make else 0)
                        card_trick_map[card] = 13-int(row[5])
        
        card_MP_map = add_MP(current_hand)
        card_imploss_map = add_imploss(current_hand)
        
        for card, val in card_score_map.items():
            card_score_map[card] = (val/counter, round(100*card_MP_map[card]/counter, 2), round(100*card_made_map[card]/counter, 2), card_trick_map[card]/counter, card_imploss_map[card]/counter)
            
        return card_score_map, counter

    # Usage
    filename = "generated_hands_solutions.csv"  # Replace with your CSV file name
    card_score_dictionary, counter = parse_csv_to_dict(filename, score_map = score_map)

    # Output the resulting dictionary
    for item in sorted([(score, card) for card, score in card_score_dictionary.items()]):
        print(item)
        
    true_list=sorted([(card, score) for card, score in card_score_dictionary.items()])

    target_file=f"/home/kstef/Desktop/BestLead/best_lead/assets/hands/{category_name}/hand{idx}.txt"

    with open(target_file, 'w') as f:
        for item in true_list:
            f.write(item[0] + ", " + str(item[1])[1:-1] + "\n")
        f.close()
        
    return


In [12]:

# compose all together
for i in range(500,501):
    print(f"generating hand no. {i}")
    #generate_pbn()
    generate_hands_given_leader(1000)

    exit_code = run_bash_script(script_path)
    print("Script exited with code:", exit_code)

    parse_solutions(category_name = "1nt-3nt", score_map=score_map_3nt, idx=i, tricks_to_make=9)                    ####CONFIG!!!!!!!!!


generating hand no. 500
['SA', 'S8', 'S4', 'HJ', 'H5', 'H2', 'DJ', 'DT', 'D5', 'D2', 'CK', 'C8', 'C7']
Generated hands saved to generated_hands.txt.
Output:
 make: 'SolveBoardPBN' is up to date.

Script exited with code: 0
((295.74, 56.66, 73.5, 9.286, 1.334), 'H2')
((295.74, 56.66, 73.5, 9.286, 1.334), 'H5')
((300.55, 55.02, 74.4, 9.321, 1.449), 'HJ')
((300.61, 52.93, 74.4, 9.327, 1.477), 'SA')
((307.4, 54.18, 76.0, 9.32, 1.602), 'S4')
((308.39, 54.0, 76.2, 9.325, 1.623), 'S8')
((327.29, 52.04, 80.0, 9.411, 2.032), 'DJ')
((327.29, 52.04, 80.0, 9.411, 2.032), 'DT')
((330.36, 48.92, 80.0, 9.51, 2.096), 'D2')
((330.36, 48.92, 80.0, 9.51, 2.096), 'D5')
((351.74, 44.28, 84.4, 9.596, 2.612), 'C7')
((351.74, 44.28, 84.4, 9.596, 2.612), 'C8')
((398.4, 30.06, 92.2, 10.05, 3.59), 'CK')


# Generate and save hands in .txt

In [2]:
default_set=[i for i in range(52)]
num_map={0:"2",1:"3",2:"4",3:"5",4:"6",5:"7",6:"8",7:"9",8:"T",9:"J",10:"Q",11:"K",12:"A"}

In [222]:
def get_PBN():
    shuffle(default_set)
    N=default_set[:13]
    E=default_set[13:26]
    S=default_set[26:39]
    W=default_set[39:]
    #print(N)
    def get_hand(l):
        sol=""
        l.sort()
        
        total_dots=0
        for i in range(12,-1,-1):
            if i<12:
                num_dots=int(l[i+1]/13)-int(l[i]/13)
                dots="".join(["." for _ in range(num_dots)])
                total_dots+=num_dots
                sol+=dots
            else:
                num_dots=3-int(l[i]/13)
                
                dots="".join(["." for _ in range(num_dots)])
                total_dots+=num_dots
                sol+=dots
            card=num_map[l[i]%13]
            sol+=card
        dots="".join(["." for _ in range(3-total_dots)])
        sol+=dots
        return sol
    return f"N:{get_hand(N)} {get_hand(E)} {get_hand(S)} {get_hand(W)}"
    

In [304]:
get_PBN()

'N:QJ63.A62.J873.JT K7.KJ54.6.Q98532 T52.973.AKT2.AK4 A984.QT8.Q954.76'

In [305]:
def generate_hands(index=0, num_boards=100000):
    all_pbns=[]
    with open(f'hands{index}.txt', 'w') as f:
        for _ in range(num_boards):
            pbn=get_PBN()
            all_pbns.append(pbn)
            f.write(pbn)
            f.write('\n')
    print(len(all_pbns))

In [306]:
for ind in range(1,65):
    generate_hands(ind)

100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000
100000


# Reading solved boards

In [323]:
def interpret_dds(ind=1):
    transformCards={"2":0,"3":1,"4":2,"5":3,"6":4,"7":5,"8":6,"9":7,"T":8,"J":9,"Q":10,"K":11, "A":12}
    transformSuit={"S":0, "H":1,"D":2,"C":3}
    
    df=pd.read_csv(f"test{ind}.csv").rename(columns={"0":"board"})
    df=df[df["rank"]!="rank"]
    
    df["equals"]=df["equals"].fillna("0")
    
    def get_index(r):
        rank, suit, equals=r
        rank, suit= transformCards[rank], transformSuit[suit]
        #print(rank+13*suit)
        indices=[rank+13*suit]
        for card in str(equals):
            if card=="0":
                continue
            tind=transformCards[card]
            indices.append(tind+13*suit)
        return indices
    
    df["indices"]=df[["rank", "suit", "equals"]].apply(lambda r: get_index(r), axis=1)
    
    inddf=df.groupby(by=["board","score"])["indices"].sum().reset_index()
    
    def tuplefy(r):
        board, score, indices=r
        sol=[]
        for i in indices:
            sol.append((i,score))
        return sol
        
    inddf["indices"]=inddf.apply(lambda r: tuplefy(r), axis=1)
    
    inddf=inddf.explode(column="indices")
    inddf["indices"]=inddf["indices"].apply(lambda ind:[ind])
    finaldf=inddf.groupby(["board"])["indices"].sum()
    
    def get_output(l):
        sol=[0 for _ in range(52)]
        for tup in l:
            sol[tup[0]]=int(tup[1])
        return sol
    
    finaldf=finaldf.apply(get_output)
    
    np.save(f"/home/kstef/Desktop/SDS/data/single_output{ind}",finaldf.apply(lambda l: max(l)).to_numpy())

In [324]:
for i in range(2,13):
    interpret_dds(i)